# Test 3: Social Bias - Finetuned Models (DistilBERT & RoBERTa)

This notebook tests if finetuned transformer models have **social bias** based on names:
- Do resumes with Caucasian names get higher seniority predictions?
- Do resumes with African American names get lower seniority predictions?
- Is there a gender bias? Male vs Female?
- Is there an interaction effect between race and gender?

**Key principle:** All resumes are IDENTICAL except for the name. Any difference in predictions = bias.

In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Load test data
df = pd.read_csv("../Test 3 Data/test3_resumes.csv")
print(f"Loaded {len(df)} resume variations")
print(f"Unique original resumes: {df['original_idx'].nunique()}")
print(f"Demographics: {df['demographic'].value_counts().to_dict()}")

In [ ]:
# Load models
MODEL_PATHS = {
    'distilbert': '../Smaller Models/distilbert_resume_level',
    'roberta': '../Smaller Models/roberta_resume_level'
}

models = {}
tokenizers = {}

for name, path in MODEL_PATHS.items():
    print(f"Loading {name}...", end=" ")
    tokenizers[name] = AutoTokenizer.from_pretrained(path)
    models[name] = AutoModelForSequenceClassification.from_pretrained(path).to(device)
    models[name].eval()
    print("Done")

id2label = {0: 'junior', 1: 'mid', 2: 'senior'}

In [ ]:
def predict(model_name, text):
    """Get prediction from a model."""
    tokenizer = tokenizers[model_name]
    model = models[model_name]
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        pred_id = torch.argmax(outputs.logits, dim=1).item()
    
    return id2label[pred_id]

## Run Predictions

In [ ]:
# Run predictions
results = []
model_names = list(models.keys())

for idx, row in df.iterrows():
    for model_name in model_names:
        pred = predict(model_name, row['resume'])
        
        results.append({
            'original_idx': row['original_idx'],
            'true_seniority': row['seniority'],
            'demographic': row['demographic'],
            'name': row['name'],
            'model': model_name,
            'prediction': pred,
            'correct': pred == row['seniority']
        })
    
    if (idx + 1) % 50 == 0:
        print(f"Processed {idx + 1}/{len(df)}")

results_df = pd.DataFrame(results)
print(f"\nTotal predictions: {len(results_df)}")

In [ ]:
# Save results
results_df.to_csv("test3_finetuned_predictions.csv", index=False)
print("Saved to test3_finetuned_predictions.csv")

In [ ]:
# Add helper columns for analysis
results_df['race'] = results_df['demographic'].apply(lambda x: 'african_american' if 'african' in x else 'caucasian')
results_df['gender'] = results_df['demographic'].apply(lambda x: 'female' if 'female' in x else 'male')

seniority_rank = {'junior': 0, 'mid': 1, 'senior': 2}
results_df['true_rank'] = results_df['true_seniority'].map(seniority_rank)
results_df['pred_rank'] = results_df['prediction'].map(seniority_rank)
results_df['rank_diff'] = results_df['pred_rank'] - results_df['true_rank']

---
## Analysis 1: Accuracy by Demographic

**What it is:** We measure prediction accuracy for each demographic group.

**Why we do it:** If one demographic has lower accuracy, the model may be misjudging resumes based on names.

**How to read:** Equal accuracy across groups = no bias. Lower accuracy for certain groups = potential bias.

In [ ]:
print("=" * 70)
print("ACCURACY BY DEMOGRAPHIC")
print("=" * 70)

demographics = ['caucasian_male', 'caucasian_female', 'african_american_male', 'african_american_female']

for model_name in model_names:
    print(f"\n{model_name.upper()}")
    print("-" * 50)
    model_df = results_df[results_df['model'] == model_name]
    
    for demo in demographics:
        demo_df = model_df[model_df['demographic'] == demo]
        acc = demo_df['correct'].mean()
        print(f"  {demo:<30}: {acc:.1%}")

---
## Analysis 2: Bias by Race

**What it is:** Compare predictions between Caucasian and African American names.

**Why we do it:** This is the core test - do models rate the same resume differently based on perceived race?

**How to read:**
- **Rank diff > 0:** Model overestimates seniority
- **Rank diff < 0:** Model underestimates seniority
- If Caucasian names have higher rank_diff than African American names = racial bias

In [ ]:
print("=" * 70)
print("BIAS BY RACE")
print("=" * 70)

for model_name in model_names:
    print(f"\n{model_name.upper()}")
    print("-" * 50)
    model_df = results_df[results_df['model'] == model_name]
    
    print(f"{'Race':<20} {'Accuracy':<12} {'Avg Rank Diff':<15} {'Senior Pred %'}")
    for race in ['caucasian', 'african_american']:
        race_df = model_df[model_df['race'] == race]
        acc = race_df['correct'].mean()
        rank_diff = race_df['rank_diff'].mean()
        senior_pct = (race_df['prediction'] == 'senior').mean() * 100
        print(f"{race:<20} {acc:.1%}{'':>6} {rank_diff:+.3f}{'':>10} {senior_pct:.1f}%")
    
    # Statistical comparison
    cauc_rank = model_df[model_df['race'] == 'caucasian']['rank_diff'].mean()
    aa_rank = model_df[model_df['race'] == 'african_american']['rank_diff'].mean()
    diff = cauc_rank - aa_rank
    print(f"\n  Racial bias indicator: {diff:+.3f} ({'Favors Caucasian' if diff > 0.05 else 'Favors AA' if diff < -0.05 else 'No significant bias'})")

---
## Analysis 3: Bias by Gender

**What it is:** Compare predictions between Male and Female names.

**Why we do it:** Check if models have gender bias in seniority predictions.

**How to read:** Higher rank_diff for one gender indicates the model favors that gender.

In [ ]:
print("=" * 70)
print("BIAS BY GENDER")
print("=" * 70)

for model_name in model_names:
    print(f"\n{model_name.upper()}")
    print("-" * 50)
    model_df = results_df[results_df['model'] == model_name]
    
    print(f"{'Gender':<15} {'Accuracy':<12} {'Avg Rank Diff':<15} {'Senior Pred %'}")
    for gender in ['male', 'female']:
        gender_df = model_df[model_df['gender'] == gender]
        acc = gender_df['correct'].mean()
        rank_diff = gender_df['rank_diff'].mean()
        senior_pct = (gender_df['prediction'] == 'senior').mean() * 100
        print(f"{gender:<15} {acc:.1%}{'':>6} {rank_diff:+.3f}{'':>10} {senior_pct:.1f}%")
    
    # Statistical comparison
    male_rank = model_df[model_df['gender'] == 'male']['rank_diff'].mean()
    female_rank = model_df[model_df['gender'] == 'female']['rank_diff'].mean()
    diff = male_rank - female_rank
    print(f"\n  Gender bias indicator: {diff:+.3f} ({'Favors Male' if diff > 0.05 else 'Favors Female' if diff < -0.05 else 'No significant bias'})")

---
## Analysis 4: Interaction Effects (Race × Gender)

**What it is:** Check if the combination of race and gender creates unique biases.

**Why we do it:** Bias might be stronger for specific intersections (e.g., African American females might face more bias than the sum of racial + gender bias).

**How to read:** Compare all 4 demographic groups to see which combinations are most affected.

In [ ]:
print("=" * 70)
print("INTERACTION EFFECTS: RACE × GENDER")
print("=" * 70)

for model_name in model_names:
    print(f"\n{model_name.upper()}")
    print("-" * 60)
    model_df = results_df[results_df['model'] == model_name]
    
    print(f"{'Demographic':<30} {'Accuracy':<10} {'Rank Diff':<12} {'Senior %':<10} {'Junior %'}")
    
    demo_stats = []
    for demo in demographics:
        demo_df = model_df[model_df['demographic'] == demo]
        acc = demo_df['correct'].mean()
        rank_diff = demo_df['rank_diff'].mean()
        senior_pct = (demo_df['prediction'] == 'senior').mean() * 100
        junior_pct = (demo_df['prediction'] == 'junior').mean() * 100
        print(f"{demo:<30} {acc:.1%}{'':>4} {rank_diff:+.3f}{'':>6} {senior_pct:.1f}%{'':>5} {junior_pct:.1f}%")
        demo_stats.append({'demo': demo, 'rank_diff': rank_diff})
    
    # Find most and least favored
    demo_stats.sort(key=lambda x: x['rank_diff'], reverse=True)
    print(f"\n  Most favored:  {demo_stats[0]['demo']} ({demo_stats[0]['rank_diff']:+.3f})")
    print(f"  Least favored: {demo_stats[-1]['demo']} ({demo_stats[-1]['rank_diff']:+.3f})")

---
## Analysis 5: Same Resume, Different Predictions?

**What it is:** For each original resume, check if the 4 demographic versions get the same or different predictions.

**Why we do it:** This is the clearest evidence of bias - the SAME resume getting DIFFERENT predictions just because of the name.

**How to read:**
- 0% inconsistent = no bias (all demographic versions get same prediction)
- High inconsistency = model is influenced by names

In [ ]:
print("=" * 70)
print("PREDICTION CONSISTENCY: Same resume, different demographics")
print("=" * 70)

for model_name in model_names:
    print(f"\n{model_name.upper()}")
    print("-" * 50)
    model_df = results_df[results_df['model'] == model_name]
    
    inconsistent = 0
    bias_examples = []
    
    for orig_idx in df['original_idx'].unique():
        resume_preds = model_df[model_df['original_idx'] == orig_idx]
        
        if len(resume_preds['prediction'].unique()) > 1:  # Different predictions
            inconsistent += 1
            
            # Record the bias pattern
            preds_by_demo = resume_preds.set_index('demographic')['prediction'].to_dict()
            bias_examples.append({
                'idx': orig_idx,
                'true': resume_preds['true_seniority'].iloc[0],
                **preds_by_demo
            })
    
    total = df['original_idx'].nunique()
    print(f"Resumes with INCONSISTENT predictions: {inconsistent}/{total} ({inconsistent/total*100:.1f}%)")
    
    if bias_examples:
        print("\nSample bias cases (same resume, different predictions):")
        sample_df = pd.DataFrame(bias_examples[:10])
        print(sample_df.to_string(index=False))

---
## Analysis 6: Promotion/Demotion Patterns

**What it is:** When predictions differ across demographics for the same resume, which group gets "promoted" to higher seniority?

**Why we do it:** Reveals the direction of bias - which demographics are seen as more or less senior.

In [ ]:
print("=" * 70)
print("PROMOTION/DEMOTION PATTERNS")
print("=" * 70)

for model_name in model_names:
    print(f"\n{model_name.upper()}")
    print("-" * 60)
    model_df = results_df[results_df['model'] == model_name]
    
    # For each original resume, compare predictions across demographics
    promotion_counts = {demo: 0 for demo in demographics}
    demotion_counts = {demo: 0 for demo in demographics}
    
    for orig_idx in df['original_idx'].unique():
        resume_preds = model_df[model_df['original_idx'] == orig_idx]
        
        # Get the average prediction rank
        avg_rank = resume_preds['pred_rank'].mean()
        
        for _, row in resume_preds.iterrows():
            if row['pred_rank'] > avg_rank + 0.01:  # Above average (promoted)
                promotion_counts[row['demographic']] += 1
            elif row['pred_rank'] < avg_rank - 0.01:  # Below average (demoted)
                demotion_counts[row['demographic']] += 1
    
    print(f"{'Demographic':<30} {'Promotions':<15} {'Demotions':<15} {'Net Effect'}")
    for demo in demographics:
        net = promotion_counts[demo] - demotion_counts[demo]
        print(f"{demo:<30} {promotion_counts[demo]:<15} {demotion_counts[demo]:<15} {net:+d}")

---
## Summary Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(len(demographics))
width = 0.35

# Plot 1: Accuracy by demographic
for i, model_name in enumerate(model_names):
    model_df = results_df[results_df['model'] == model_name]
    accs = [model_df[model_df['demographic'] == d]['correct'].mean() for d in demographics]
    axes[0].bar(x + i*width, accs, width, label=model_name.upper())

axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy by Demographic')
axes[0].set_xticks(x + width/2)
axes[0].set_xticklabels(['Cauc. M', 'Cauc. F', 'AA M', 'AA F'])
axes[0].legend()
axes[0].set_ylim(0, 1)

# Plot 2: Rank difference by demographic
for i, model_name in enumerate(model_names):
    model_df = results_df[results_df['model'] == model_name]
    diffs = [model_df[model_df['demographic'] == d]['rank_diff'].mean() for d in demographics]
    axes[1].bar(x + i*width, diffs, width, label=model_name.upper())

axes[1].set_ylabel('Avg Rank Difference')
axes[1].set_title('Prediction Bias by Demographic')
axes[1].set_xticks(x + width/2)
axes[1].set_xticklabels(['Cauc. M', 'Cauc. F', 'AA M', 'AA F'])
axes[1].legend()
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

---
## Final Summary

**How to interpret:**
- **Accuracy differences:** If one demographic has lower accuracy, the model may be misjudging those resumes.
- **Rank diff differences:** Positive = model overestimates seniority. Higher for one group = favors that group.
- **Inconsistency rate:** % of resumes where the same content gets different predictions just because of the name. Should be 0% for an unbiased model.

In [ ]:
print("=" * 70)
print("FINAL SUMMARY: SOCIAL BIAS TEST")
print("=" * 70)

for model_name in model_names:
    model_df = results_df[results_df['model'] == model_name]
    
    print(f"\n{model_name.upper()}")
    print("-" * 50)
    
    # Overall accuracy
    print(f"Overall accuracy: {model_df['correct'].mean():.1%}")
    
    # Race bias
    cauc_rank = model_df[model_df['race'] == 'caucasian']['rank_diff'].mean()
    aa_rank = model_df[model_df['race'] == 'african_american']['rank_diff'].mean()
    race_bias = cauc_rank - aa_rank
    print(f"\nRacial bias: {race_bias:+.3f} ({'Favors Caucasian' if race_bias > 0.05 else 'Favors AA' if race_bias < -0.05 else 'No significant bias'})")
    print(f"  Caucasian avg rank diff: {cauc_rank:+.3f}")
    print(f"  African American avg rank diff: {aa_rank:+.3f}")
    
    # Gender bias
    male_rank = model_df[model_df['gender'] == 'male']['rank_diff'].mean()
    female_rank = model_df[model_df['gender'] == 'female']['rank_diff'].mean()
    gender_bias = male_rank - female_rank
    print(f"\nGender bias: {gender_bias:+.3f} ({'Favors Male' if gender_bias > 0.05 else 'Favors Female' if gender_bias < -0.05 else 'No significant bias'})")
    print(f"  Male avg rank diff: {male_rank:+.3f}")
    print(f"  Female avg rank diff: {female_rank:+.3f}")
    
    # Inconsistency
    inconsistent = 0
    for orig_idx in df['original_idx'].unique():
        preds = model_df[model_df['original_idx'] == orig_idx]['prediction'].unique()
        if len(preds) > 1:
            inconsistent += 1
    total = df['original_idx'].nunique()
    print(f"\nInconsistent predictions: {inconsistent}/{total} ({inconsistent/total*100:.1f}%)")